# Imports

In [1]:
# import time
from technician_routing.routing_toolkit import RoutingConfig, RoutingInterface
from technician_routing.routing_toolkit.engine import create_search_parameters

# Demo

#### Configure and Load Data

In [5]:
config = RoutingConfig(
    'u:/data/darnley_tech_address.csv', 
    'u:/data/address_lat_long_121923.csv',
    40, # MPH
    55, # Minutes Per Service
    60*7 # Max Minutes per Route
)
interface = RoutingInterface(config).load_data()

Set 5269 client addresses
Using uniform minutes on location of 55
Data Loaded


#### Choose your clients

In [7]:
focus_addresses = interface.client_addresses[0:1000]
print(len(focus_addresses))

1000


#### Choose your technicians

In [9]:
#drivers = 25 # Weighted-Random Sample
drivers = [0,1,2,3,4,5,6] * 40 # Four of each technician
print(len(drivers))

280


#### Optionally Ovewrite certain defaults at runtime

In [10]:
interface \
    .set_driver_speed(45) \
    .set_addresses(focus_addresses) \
    .set_time_on_location(65) \
    .set_search_parameters(create_search_parameters(search_time_seconds=45))

Using driver speed of 45 MPH
Set 1000 client addresses
Using uniform minutes on location of 65


#### Run

In [12]:
# Random Drivers
routes = interface.run(route_drivers=drivers)
routes

[0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]
Calculated 1007 node durations in 1.4486331939697266
Running routing for 1007 addresses and 280 drivers
Solved objective value 71680
Skipped unused route 0 sta

[[1, 750, 266, 72, 876, 859, 848, 1],
 [2, 862, 728, 2],
 [3, 835, 661, 102, 488, 910, 3],
 [5, 775, 535, 968, 113, 975, 5],
 [6, 67, 231, 368, 268, 128, 111, 6],
 [2, 31, 279, 760, 751, 745, 2],
 [4, 250, 438, 504, 264, 4],
 [5, 473, 169, 704, 133, 515, 5],
 [6, 444, 365, 70, 578, 385, 130, 6],
 [4, 14, 889, 172, 983, 999, 433, 4],
 [5, 384, 75, 301, 993, 493, 5],
 [6, 298, 284, 255, 58, 6],
 [4, 218, 703, 361, 249, 53, 30, 4],
 [5, 234, 532, 527, 513, 511, 57, 5],
 [6, 486, 278, 556, 891, 617, 966, 6],
 [4, 536, 66, 633, 984, 16, 4],
 [5, 373, 200, 174, 317, 118, 5],
 [6, 712, 903, 26, 125, 940, 6],
 [4, 485, 484, 550, 963, 653, 4],
 [5, 350, 73, 512, 294, 490, 5],
 [6, 160, 366, 822, 355, 52, 6],
 [5, 668, 607, 303, 623, 419, 5],
 [6, 19, 378, 926, 173, 134, 6],
 [6, 393, 753, 381, 450, 791, 6],
 [6, 223, 399, 818, 150, 97, 6],
 [4, 858, 251, 306, 687, 4],
 [5, 152, 466, 539, 698, 5],
 [2, 935, 810, 116, 803, 453, 2],
 [3, 928, 952, 502, 107, 3],
 [4, 991, 394, 247, 708, 706, 4],
 [

#### Detail Routes

In [ ]:
for route_number, route in enumerate(routes):
    
    travel_time = 0
    for start,end in zip(route, route[1:]):
        travel_time += interface.solver.distances[start, end]

    site_time = 0
    for location in route[1:-1]:
        site_time += interface.solver.demands[location]

    tech = interface.df_techs.iloc[route[0], :]
    #print(tech)
     
    print(f'Route {route_number}: {int(travel_time + site_time)}','minutes')
    print(f'Technician {route[0]}: {tech["Tech"]} - {tech["address"]}')

    for stop in route[1:-1]:
        print(f'\t{stop}: {interface.last_addresses[stop]}')
    

    print(int(travel_time), 'travel minutes') 
    print(site_time, 'minutes on site')
    print('\n')